In [1]:
!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import datetime # modified

In [2]:
# modified

def normalize_data(df):
    # df on input should contain only one column with the price data (plus dataframe index)
    min = df.min()
    max = df.max()
    x = df 
    
    # time series normalization part
    # y will be a column in a dataframe
    y = (x - min) / (max - min)
    
    return y

In [3]:
tickers = pd.read_csv('/content/sample_data/symbols.csv')
print(tickers)
tickers= list(tickers.Tickers_yfinance)

     Tickers_yfinance  Start_Date
0        20MICRONS.NS  2008-10-06
1       21STCENMGM.NS  2018-01-11
2       3IINFOTECH.NS  2005-04-22
3          3MINDIA.NS  2002-07-01
4           3PLAND.NS  2008-03-11
...               ...         ...
1573     ZODJRDMKJ.NS  2002-07-01
1574          ZOTA.NS  2017-05-10
1575         ZUARI.NS  2012-11-27
1576     ZUARIGLOB.NS  2002-07-01
1577     ZYDUSWELL.NS  2009-11-13

[1578 rows x 2 columns]


In [4]:
def dropDates(a, b):
  ## Remove dates in a but not in b
  for i in a.index:
    if i not in b.index:
      a.drop(i,axis=0,inplace=True)
  ## Remove dates in b but not in a
  for i in b.index:
    if i not in a.index:
      b.drop(i,axis=0,inplace=True)

In [5]:
# Interpolation

def modify_stock(pr):
  pr.reset_index(level=0, inplace=True)
  def imput_date(df:pd.DataFrame,col:'Date'):
    start = datetime.datetime.strptime(df.loc[0,col],'%Y-%m-%d')
    end = datetime.datetime.strptime(df.loc[df.shape[0] - 1,col],'%Y-%m-%d')
    list_date = pd.date_range(start,end).strftime('%Y-%m-%d').tolist()
    pd_date = pd.DataFrame(df,list_date)
    pd_date[col] = pd_date.index.astype(object)
    pd_date = pd_date.reset_index(drop=True)
    df_date = pd.merge(pd_date[col],df,on=col,how='left')
    return df_date

  # Curve function
  def curve_function(df):
      for i in df.columns:
          while df[i].isna().sum() > 0:
              for j in range(df.shape[0]):
                  if pd.isnull(df.loc[j,i]):
                      seq_k = [j]
                      k = j
                      while pd.isnull(df.loc[k,i]):
                          k = k + 1
                          seq_k.append(k)
                      if len(seq_k) % 2 == 0:
                          df.loc[seq_k[int((len(seq_k) - 1)/2)],i] = (df.loc[j - 1,i] + df.loc[seq_k[len(seq_k) - 1],i])/2
                      else:
                          df.loc[seq_k[int((len(seq_k) - 1)/2)],i] = (df.loc[j - 1,i] + df.loc[seq_k[len(seq_k) - 1],i])/2
                  else:
                      df.loc[j,i] = df.loc[j,i]
      return(df)

  pr['Date'] = pr['Date'].dt.strftime('%Y-%m-%d')
  pr = pr.pipe(imput_date,col='Date')
  # Create dummy variable for deterimining free day or not
  free = []
  for i in range(pr.shape[0]):
      if pd.isna(pr.iloc[i]['Volume']):
          free.append(0)
      else:
          free.append(1)
  pr_n = pr.pipe(curve_function)
  return(pr_n)

In [6]:
def SSD(stocka, stockb):
  stock1 = modify_stock(stocka)  # modified
  stock2 = modify_stock(stockb)  # modified
  stock1['norm'] = normalize_data(stock1['Close'])   # modified
  stock2['norm'] = normalize_data(stock2['Close'])   # modified
  dropDates(stock1, stock2)
  len1 = len(stock1.index)
  len2 = len(stock2.index)
  idx = 252
  try:
    buy1 = stock1.norm[len1 - 252]
  except:
    print('Not enough data')
    return -1
  try:
    buy2 = stock2.norm[len2 - 252]
  except:
    print('Not enough data')
    return -1
  p1 = 1
  p2 = 1
  ssd = 0
  while(idx > 0):
    sell1 = stock1.norm[len1 - idx]
    sell2 = stock2.norm[len2 - idx]
    r1 = (sell1 - buy1) / buy1
    r2 = (sell2 - buy2) / buy2
    p1 *= (1 + r1)
    p2 *= (1 + r2)
    ssd += (p1 - p2) * (p1 - p2)
    idx -= 1
  print(ssd)
  return ssd

In [7]:
def find_pairs():
  msft = yf.Ticker('MSFT').history(start="2018-01-01",  end="2020-11-30")

  min = SSD(msft, yf.Ticker(tickers[0]).history(start="2018-01-01",  end="2020-11-30"))
  if min == -1:
    return
  print(type(min))
  pair = ""
  for ticker in tickers:
    msft = yf.Ticker('MSFT').history(start="2018-01-01",  end="2020-11-30")
    print(ticker)
    target = yf.Ticker(ticker).history(start="2018-01-01",  end="2020-11-30")
    s = SSD(msft, target)
    if s == -1:
      continue
    if s < min:
      min=s
      pair = ticker
  print (min, pair)

In [8]:
find_pairs()

3.171411183969119e+160
<class 'numpy.float64'>
20MICRONS.NS
3.171411183969119e+160
21STCENMGM.NS
3.7489084104243488e+146
3IINFOTECH.NS
3.171411183969119e+160
3MINDIA.NS
3.171411183969119e+160
3PLAND.NS
3.171411183969119e+160
5PAISA.NS
3.171411183969119e+160
63MOONS.NS
3.171411183969119e+160
A2ZINFRA.NS
3.171411183969119e+160
AARTIDRUGS.NS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in double_scalars


inf
AARTIIND.NS
3.171411183969119e+160
AARVEEDEN.NS
3.171411183969119e+160
AARVI.NS
3.171411183969119e+160
AAVAS.NS
2.867350957081031e+56
ABAN.NS
3.171411183969119e+160
ABB.NS
3.171411183969119e+160
ABBOTINDIA.NS
3.171411183969119e+160
ABCAPITAL.NS
3.171411183969119e+160
ABFRL.NS
3.171411183969119e+160
ABMINTLTD.NS
3.171411183969119e+160
ACC.NS
3.171411183969119e+160
ACCELYA.NS
3.171411183969119e+160
ACE.NS
3.171411183969119e+160
ADANIENT.NS
3.171411183969029e+160
ADANIGREEN.NS
2.6103997495881116e+250
ADANIPORTS.NS
3.171411183969119e+160
ADANIPOWER.NS
3.171411183969119e+160
ADANITRANS.NS
3.171411183969119e+160
ADFFOODS.NS
3.171411183969119e+160
ADHUNIKIND.NS
3.171411183969119e+160
ADORWELD.NS
3.171411183969119e+160
ADROITINFO.NS
5.875401412787267e+267
ADSL.NS
2.06336675160905e+301
ADVANIHOTR.NS
3.171411183969119e+160
ADVENZYMES.NS
3.171411183969119e+160
AEGISCHEM.NS
3.171411183969119e+160
AFFLE.NS
7.547426013453811e+232
AGARIND.NS
3.171411183969119e+160
AGCNET.NS
3.171411183969119e+160

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


inf
ALEMBICLTD.NS
3.171411183969119e+160
ALICON.NS
3.171411183969119e+160
ALKALI.NS
3.171411183969119e+160
ALKEM.NS
3.171411183969119e+160
ALKYLAMINE.NS
2.6271029392152477e+195
ALLCARGO.NS
3.171411183969119e+160
ALLSEC.NS
3.171411183969119e+160
ALMONDZ.NS
3.171411183969119e+160
ALOKINDS.NS


...

nan
VIVIMEDLAB.NS
3.171411183969119e+160
VLSFINANCE.NS
3.171411183969119e+160
VMART.NS
3.171411183969119e+160
VOLTAMP.NS
3.171411183969119e+160
VOLTAS.NS
3.171411183969119e+160
VRLLOG.NS
3.171411183969119e+160
VSSL.NS
3.171411183969119e+160
VSTIND.NS
3.171411183969119e+160
VSTTILLERS.NS
3.171411183969119e+160
VTL.NS
3.171411183969119e+160
WABAG.NS
3.171411183969119e+160
WABCOINDIA.NS
3.171411183969119e+160
WALCHANNAG.NS
3.171411183969119e+160
WANBURY.NS
1.2579340320898645e+177
WATERBASE.NS
441209897239.18115
WEBELSOLAR.NS
3.171411183969119e+160
WEIZMANIND.NS
5.1813517325928245e+205
WELCORP.NS
3.171411183969119e+160
WELENT.NS
3.171411183969119e+160
WELINV.NS
3.1714111839688785e+160
WELSPUNIND.NS
3.171411183969119e+160
WENDT.NS
3.171411183969119e+160
WESTLIFE.NS
9.199278133084559e+80
WHEELS.NS
3.171411183969119e+160
WHIRLPOOL.NS
3.171411183969119e+160
WILLAMAGOR.NS
3.171411183969119e+160
WINDMACHIN.NS
3.171411183969119e+160
WIPL.NS
5.537526713911721e+217
WIPRO.NS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


inf
WOCKPHARMA.NS
3.171411183969119e+160
WONDERLA.NS
3.171411183969119e+160
WSI.NS
3.171411183969119e+160
WSTCSTPAPR.NS
3.171411183969119e+160
XCHANGING.NS
3.171411183969119e+160
XELPMOC.NS
1.780332518812873e+137
XPROINDIA.NS
3.171411183969119e+160
YESBANK.NS
3.171411183969119e+160
ZEEL.NS
3.171411183969119e+160
ZEELEARN.NS
3.171411183969119e+160
ZEEMEDIA.NS
3.171411183969119e+160
ZENITHEXPO.NS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


inf
ZENSARTECH.NS
3.171411183969119e+160
ZENTEC.NS
3.171411183969119e+160
ZICOM.NS
3.171411183969119e+160
ZODIACLOTH.NS
3.171411183969119e+160
ZODJRDMKJ.NS
3.171411183969119e+160
ZOTA.NS
3.171411183969119e+160
ZUARI.NS
3.171411183969119e+160
ZUARIGLOB.NS
3.171411183969119e+160
ZYDUSWELL.NS
3.1714111839653946e+160
0.7159160824526132 MAHEPC.NS
